# ❄️ Anthropic on Snowflake Cortex – End‑to‑End RAG Notebook
This single notebook:
1. Sets up a Snowflake RAG environment
2. Ingests PDF & CSV docs
3. Builds a Cortex Search Service
4. Loads a Standards Library from Excel
5. Classifies each control as **Yes / No / Needs Review**
6. (Optional) Deploys a Streamlit dashboard for human QA

Cells tagged **SQL** use the notebook's SQL runner; others run in Python.

In [ ]:

-- SECTION 0 – Prerequisites  (SQL)
USE ROLE ACCOUNTADMIN;
CREATE DATABASE IF NOT EXISTS corp_rag;
CREATE SCHEMA   IF NOT EXISTS corp_rag;
USE DATABASE corp_rag;
USE SCHEMA   corp_rag;


## 1  Python environment

In [ ]:

# Ensure the notebook packages via the *Packages* pane:
#   snowflake-ml-python>=1.8, snowflake, streamlit
import pandas as pd, json, uuid, streamlit as st
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import col
from snowflake.cortex import complete
from snowflake.core import Root

session       = get_active_session()
root          = Root(session)
warehouse     = session.get_current_warehouse()
database_name = session.get_current_database()
schema_name   = session.get_current_schema()
service_name  = "document_search_service"


## 2  Stage & list source files

In [ ]:

CREATE STAGE IF NOT EXISTS docs_stage DIRECTORY = (ENABLE = TRUE);
-- Drag & drop PDFs/CSVs into corp_rag ▸ docs_stage ▸ Upload
LIST @docs_stage;


## 3  Parse PDFs

In [ ]:

from snowflake.snowpark.types import StringType
pdf_files = [r["name"].split("/")[1] for r in session.sql("LIST @docs_stage/*.pdf").collect()]

def parse_pdf(fname):
    q = """SELECT TO_VARCHAR(
               SNOWFLAKE.CORTEX.PARSE_DOCUMENT(@docs_stage, ?, {'mode':'OCR'}):content
             ) AS text;"""
    return session.sql(q, params=[fname]).collect()[0]["TEXT"].replace("'", "")

pdf_df = pd.DataFrame({"file_name": pdf_files,
                       "text": [parse_pdf(f) for f in pdf_files]})


## 4  Load CSVs

In [ ]:

session.sql("""CREATE OR REPLACE FILE FORMAT csv_fmt
                 TYPE = CSV FIELD_OPTIONALLY_ENCLOSED_BY='"' SKIP_HEADER=1""").collect()
csv_snow = (session.read.schema("variant")
                     .option("file_format","csv_fmt")
                     .csv("@docs_stage/*.csv"))
csv_flat = csv_snow.select(col("$1").cast(StringType()).alias("text"),
                           col("METADATA$FILENAME").alias("file_name"))
csv_df = csv_flat.to_pandas()
full_df = pd.concat([pdf_df, csv_df], ignore_index=True)


## 5  Persist docs_text_table

In [ ]:

(session.create_dataframe(full_df)
        .select(col("file_name"), col("text"))
        .write.mode("overwrite")
        .save_as_table("docs_text_table"))


## 6  Create / refresh Cortex Search Service

In [ ]:

CREATE OR REPLACE CORTEX SEARCH SERVICE document_search_service
  ON text
  ATTRIBUTES file_name
  WAREHOUSE = ${warehouse}
  TARGET_LAG = '1 day'
  EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
  AS (
    SELECT text, file_name FROM docs_text_table
  );


## 7  Standards Library skeleton

In [ ]:

CREATE OR REPLACE TABLE standards (
  id        INT AUTOINCREMENT,
  standard  STRING,
  answer    STRING,
  rationale STRING
);
-- Example seeds
INSERT INTO standards (standard) VALUES
 ('Does the report include an ISO‑27001 certificate?'),
 ('Is PII encrypted at rest?'),
 ('Is SOC 2 Type II coverage current?');


## 8  Classify each standard

In [ ]:

MODEL = "claude-3-5-sonnet"
svc = (root.databases[database_name]
           .schemas[schema_name]
           .cortex_search_services[service_name])
def classify_row(row):
    req = row.STANDARD
    ctx = svc.search(query=req, columns=["text"], limit=6).to_pandas()["text"].str.cat(sep="\n\n")
    prompt = f"""You are an auditor. Decide if <context> proves <requirement> is met. 
Return JSON {{\"answer\":\"Yes|No|Needs Review\",\"rationale\":"one line"}}.
<requirement>{req}</requirement>
<context>{ctx}</context>"""
    j = json.loads(complete(MODEL, prompt, temperature=0))
    return (row.ID, j["answer"], j["rationale"])

## 9  Streamlit review dashboard (optional)

In [ ]:

# Save as streamlit/review_app.py if desired
review_code = """import streamlit as st
from snowflake.snowpark.context import get_active_session
session = get_active_session()
df = session.table('standards').filter("answer IS NULL OR answer='Needs Review'").to_pandas()
st.title('🕵️ Needs‑Review Dashboard')
edited = st.data_editor(df, num_rows='dynamic',
    column_config={'answer': st.column_config.Selectbox('answer', options=['Yes','No','Needs Review'])})
if st.button('Save changes'):
    session.write_pandas(edited, 'tmp_review', auto_create_table=True)
    session.sql(\"\"\"MERGE INTO standards t USING tmp_review s ON s.ID=t.ID
                 WHEN MATCHED THEN UPDATE SET answer=s.answer, rationale=s.rationale\"\"\").collect()
    st.success('Changes saved')
"""

## 11  Load StandardsLibrary.xlsx

In [ ]:

CREATE STAGE IF NOT EXISTS standards_stage DIRECTORY = (ENABLE = TRUE);
-- Upload StandardsLibrary.xlsx to standards_stage
LIST @standards_stage;

CREATE OR REPLACE FILE FORMAT excel_fmt TYPE = 'EXCEL';

CREATE OR REPLACE TABLE standards_library_raw AS
SELECT *
FROM @standards_stage (FILE_FORMAT => excel_fmt);


## 12  Normalize Yes / No / Needs Review columns

In [ ]:

raw = session.table("standards_library_raw").to_pandas()
questions = raw[ raw["Type"].str.lower() == "question" ]
norm = (questions.rename(columns={"ID":"id","Name":"standard"})
        [["id","standard","Yes","No","Needs Review"]])
(session.create_dataframe(norm)
        .write.mode("overwrite")
        .save_as_table("standards"))
print(f"{len(norm)} standards loaded to table.")


## 13  Classify unanswered rows and set flags

In [ ]:

target = session.table("standards").filter("Yes IS NULL AND No IS NULL AND `Needs Review` IS NULL").to_pandas()
updates = pd.DataFrame([classify_row(r) for _, r in target.iterrows()],
                       columns=["ID","ANSWER","RATIONALE"])
session.write_pandas(updates, "tmp_updates", auto_create_table=True)
session.sql("""MERGE INTO standards t USING tmp_updates s ON s.ID = t.ID
WHEN MATCHED THEN UPDATE SET
  Yes           = IFF(s.answer='Yes','X',NULL),
  No            = IFF(s.answer='No','X',NULL),
  `Needs Review`= IFF(s.answer='Needs Review','X',NULL),
  rationale     = s.rationale;
""").collect()
print("Classification complete.")
